In [5]:
import numpy as np
import piquasso as pq

import itertools
import functools

from typing import List

np.set_printoptions(linewidth=200, suppress=True)


updates = 10

shots = 5

M = d = 5  # modes

eta = learning_rate = 0.3


# Ising model

J = 2 * np.random.rand(d, d) - 1  # Interaction

h = np.random.rand(d)  # External magnetic field

mu = np.random.rand()  # Magnetic moment

# Map to QUBO

Q_diagonal = np.zeros(shape=(d,))

for index in range(len(J)):
    # The summation may be modified to only permit interaction between neighbouring
    # cells.
    Q_diagonal[index] = 2 * (
        sum(J[index, :])
        + sum(J[:, index])
        + mu * h[index]
    )

Q = -4 * J

np.fill_diagonal(Q, Q_diagonal)


simulator = pq.SamplingSimulator(d=d)


def calculate_QUBO_explicitely(Q):
    d = len(Q)

    bitstrings = list(map(np.array, list(itertools.product([0, 1], repeat=d))))

    values = []

    for bitstring in bitstrings:
        values.append(bitstring @ Q @ bitstring)

    return min(values), bitstrings[np.argmin(values)]



def map_to_bitstring(sample: np.ndarray, parity: int) -> np.ndarray:
    return list(map(lambda x: (x + parity) % 2, sample))


def get_samples(thetas: List[float]):
    iterator = iter(thetas)

    with pq.Program() as program:
        pq.Q(all) | pq.StateVector((1, ) * d)

        for column in range(d):
            start_index = column % 2
            for element in range(start_index, d - 1, 2):
                pq.Q(element, element + 1) | pq.Beamsplitter(next(iterator))

        pq.Q() | pq.Sampling()

    return simulator.execute(program, shots).samples


min_energy = None


thetas = np.random.uniform(0.0, 2 * np.pi, d * (d - 1) // 2)

def get_energy(thetas: List[float], mapper) -> float:
    samples = get_samples(thetas)
    bitstrings = list(map(mapper, samples))
    energies = [bitstring @ Q @ bitstring for bitstring in bitstrings]
    return sum(energies) / shots


def update_thetas(thetas: List[float], mapper):
    derivatives = []

    new_thetas = np.copy(thetas)

    for j in range(len(thetas)):
        upshifted_thetas = np.copy(thetas)
        upshifted_thetas[j] += np.pi / 2
        upshifted_energy = get_energy(upshifted_thetas, mapper=map_to_bistring_with_parity)

        downshifted_thetas = np.copy(thetas)
        downshifted_thetas[j] -= np.pi / 2
        downshifted_energy = get_energy(upshifted_thetas, mapper=map_to_bistring_with_parity)

        derivative = (upshifted_energy - downshifted_energy) / 2

        new_thetas[j] -= eta * derivative

    return new_thetas


for parity in (0, 1):
    map_to_bistring_with_parity = functools.partial(map_to_bitstring, parity=parity)
    for _ in range(updates):

        samples = get_samples(thetas)

        bitstrings = list(map(map_to_bistring_with_parity, samples))
        energies = [bitstring @ Q @ bitstring for bitstring in bitstrings]
        energy_mean = sum(energies) / shots

        print("Energies:", energies)

        if min_energy is None or min(energies) < min_energy:
            min_energy = min(energies)
            min_bistring = bitstrings[np.argmin(energies)]

        thetas = update_thetas(thetas, map_to_bistring_with_parity)


print("Algo:", min_energy, min_bistring)

print("Exact solution:", calculate_QUBO_explicitely(Q))

# NOTE: An offset is needed to be added to the energy for the Ising solution.

Energies: [-0.9650903680514537, -6.189401572915884, -0.7977514403868415, -0.9650903680514537, 4.957147608995949]
Energies: [4.957147608995949, 0.230590527822514, 0.9417620560209968, -1.3261477701099187, -6.189401572915884]
Energies: [-0.7977514403868415, -0.7977514403868415, 0.230590527822514, 1.751733856519297, 0.9417620560209968]
Energies: [0.9417620560209968, -6.189401572915884, 0.9417620560209968, -0.7977514403868415, -6.189401572915884]
Energies: [0.15790010325755022, -0.9650903680514537, 0.230590527822514, -1.2536762585396466, -0.7977514403868415]
Energies: [-2.1589659571789697, -5.864730734981972, -5.864730734981972, -5.864730734981972, -2.1589659571789697]
Energies: [0.230590527822514, -1.3261477701099187, 1.751733856519297, 1.751733856519297, -1.3261477701099187]
Energies: [0.230590527822514, -1.2536762585396466, -1.2536762585396466, -1.2536762585396466, 0.230590527822514]
Energies: [-1.3261477701099187, -1.2536762585396466, -1.2536762585396466, -1.3261477701099187, -1.2536762